# Solving Sudoku with JuMP - A short version
* The numbers 1 to 9 must appear in each 3x3 square
* The numbers 1 to 9 must appear in each row
* The numbers 1 to 9 must appear in each column

Lets pose this as a mixed integer feasibility problem in JuMP!


In [13]:
using JuMP, GLPK
# Create a model
sudoku = Model(with_optimizer(GLPK.Optimizer));

### Create 3 dimensional, binary array `x`

`x[i,j,k] = 1`  if and only if cell `(i,j)` has number `k`

In [2]:
@variable(sudoku, x[i=1:9, j=1:9, k=1:9], Bin);

### Each cell can only contain one number

In [3]:
@constraint(sudoku, cell[i=1:9, j=1:9], sum(x[i,j,1:9]) == 1);

### Rows and columns should contain each number once 

In [4]:
@constraint(sudoku, col[i=1:9, k=1:9], sum(x[i,1:9,k]) == 1)
@constraint(sudoku, row[j=1:9, k=1:9], sum(x[1:9,j,k]) == 1);

### Each box should contain each number once

In [5]:
@constraint(sudoku, box[i=1:3:7, j=1:3:7, k=1:9], sum(x[i:i+2,j:j+2,k]) == 1);

### Let's take an initial guess

In [6]:
# The given digits
init_sol = [ 5 3 0 0 7 0 0 0 0;
             6 0 0 1 9 5 0 0 0;
             0 9 8 0 0 0 0 6 0;
             8 0 0 0 6 0 0 0 3;
             4 0 0 8 0 3 0 0 1;
             7 0 0 0 2 0 0 0 6;
             0 6 0 0 0 0 2 8 0;
             0 0 0 4 1 9 0 0 5;
             0 0 0 0 8 0 0 7 9];

### And set the constraints

In [7]:
# All indices where we have initial constraints
ij = [(i,j) for i =1:9 for j=1:9 if init_sol[i,j] != 0]

@constraint(sudoku, init[(i,j) in ij], x[i,j,init_sol[i,j]] == 1);

### We are now ready to solve the problem

In [8]:
@time optimize!(sudoku)
termination_status(sudoku)

 14.036755 seconds (36.49 M allocations: 1.815 GiB, 7.61% gc time)


OPTIMAL::TerminationStatusCode = 1

### Extract the values of x

In [9]:
x_val = round.(Int,value.(x))
# Create a matrix to store the solution
sol = zeros(Int,9,9)  # 9x9 matrix of integers
for i in 1:9, j in 1:9, k in 1:9
    if x_val[i,j,k] == 1
        sol[i,j] = k
    end
end

In [10]:
sol

9×9 Array{Int64,2}:
 5  3  4  6  7  8  9  1  2
 6  7  2  1  9  5  3  4  8
 1  9  8  3  4  2  5  6  7
 8  5  9  7  6  1  4  2  3
 4  2  6  8  5  3  7  9  1
 7  1  3  9  2  4  8  5  6
 9  6  1  5  3  7  2  8  4
 2  8  7  4  1  9  6  3  5
 3  4  5  2  8  6  1  7  9

### Verify

In [11]:
check(constraint) = all(isequal(1), value.(constraint))

check(cell), check(col), check(row), check(box)

(true, true, true, true)

In [12]:
all(init_sol[i,j] == sol[i,j] for (i,j) in ij)

true